# Hotel Matching & Canonical ID System
## MLOps Assignment - Amex GBT

### 1. Problem Statement:
- Business context: Travel company aggregating hotel inventory
- Challenge: Same hotel appears with different names/addresses across sources
- Goal: Build deduplication system with canonical hotel IDs

### 2. Data Loading & Initial Exploration

In [38]:
import pandas as pd
import numpy as np
import re
from sklearn.cluster import DBSCAN

In [6]:
df = pd.read_csv('hoteldata_raw.csv')

In [7]:
df.head()

,record_id,source_system,hotel_name,address_line1,address_line2,city,state,country,postal_code,phone_number
0,R001,GDS_A,Marriott Bangalore Outer Ring Road,Outer Ring Road,Doddanekundi,Bangalore,Karnataka,IN,560103,-12345667
1,R002,OTA_B,Marriott ORR Blr - Outer Ring Rd,Outer Ring Rd,NaN,Bengaluru,Karnataka,India,560103,-12345667
2,R003,DIRECT_C,Hotel Marriot Bengaluru ORR,Outer Ring Road,NaN,Bangalore,KA,IN,560103,-12345667
3,R004,GDS_A,Holiday Inn Express Whitefield,ITPL Main Road,NaN,Bangalore,Karnataka,IN,560066,-99998877
4,R005,OTA_B,Holiday Inn Exp. Bengaluru Whitefield,ITPL Rd,Whitefield,Bengaluru,KA,India,560066,-99998877


In [8]:
df.shape

(100, 10)

In [9]:
df.dtypes

record_id        object
source_system    object
hotel_name       object
address_line1    object
address_line2    object
city             object
state            object
country          object
postal_code      object
phone_number      int64
dtype: object

In [10]:
df.isnull().sum()

record_id         0
source_system     0
hotel_name        0
address_line1     0
address_line2    64
city              0
state             0
country           0
postal_code       0
phone_number      0
dtype: int64

In [12]:
# Check negative values
negative_count = (df['phone_number'] < 0).sum()
print(f"Negative phone numbers: {negative_count}")

Negative phone numbers: 25


### 3. Data Cleaning & Normalization

In [13]:
# Clean phone
df['phone_clean'] = df['phone_number'].apply(lambda x: str(abs(int(x))))
print("Sample after cleaning:")
print(df[['phone_number', 'phone_clean']].head())

Sample after cleaning:
   phone_number phone_clean
0     -12345667    12345667
1     -12345667    12345667
2     -12345667    12345667
3     -99998877    99998877
4     -99998877    99998877


In [14]:
df.duplicated().sum()

np.int64(0)

In [15]:
df.head()

,record_id,source_system,hotel_name,address_line1,address_line2,city,state,country,postal_code,phone_number,phone_clean
0,R001,GDS_A,Marriott Bangalore Outer Ring Road,Outer Ring Road,Doddanekundi,Bangalore,Karnataka,IN,560103,-12345667,12345667
1,R002,OTA_B,Marriott ORR Blr - Outer Ring Rd,Outer Ring Rd,NaN,Bengaluru,Karnataka,India,560103,-12345667,12345667
2,R003,DIRECT_C,Hotel Marriot Bengaluru ORR,Outer Ring Road,NaN,Bangalore,KA,IN,560103,-12345667,12345667
3,R004,GDS_A,Holiday Inn Express Whitefield,ITPL Main Road,NaN,Bangalore,Karnataka,IN,560066,-99998877,99998877
4,R005,OTA_B,Holiday Inn Exp. Bengaluru Whitefield,ITPL Rd,Whitefield,Bengaluru,KA,India,560066,-99998877,99998877


In [17]:
#Function to return Unique values in each column
def get_unique(df):
    for col in df:
        print(f"{col} : {df[col]}") 
        print("=========================")

In [18]:
get_unique(df)

record_id : 0     R001
1     R002
2     R003
3     R004
4     R005
      ... 
95    R096
96    R097
97    R098
98    R099
99    R100
Name: record_id, Length: 100, dtype: object
source_system : 0        GDS_A
1        OTA_B
2     DIRECT_C
3        GDS_A
4        OTA_B
        ...   
95       OTA_B
96       GDS_A
97       GDS_A
98       OTA_B
99       OTA_B
Name: source_system, Length: 100, dtype: object
hotel_name : 0        Marriott Bangalore Outer Ring Road
1          Marriott ORR Blr - Outer Ring Rd
2               Hotel Marriot Bengaluru ORR
3            Holiday Inn Express Whitefield
4     Holiday Inn Exp. Bengaluru Whitefield
                      ...                  
95             Ibis Los Angeles City Center
96             Hotel Ibis Los Angeles Cntrl
97                  DoubleTree Doha Central
98              DoubleTree Doha City Center
99              Hotel DoubleTree Doha Cntrl
Name: hotel_name, Length: 100, dtype: object
address_line1 : 0       Outer Ring Road
1         Ou

### 4. Feature Engineering

In [20]:
#Function to clean
def simple_clean(text):
    if pd.isna(text):
        return ""
    text = str(text).lower().strip()
    text = re.sub(r'[^\w\s]', ' ', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text)      # remove extra spaces
    return text

In [23]:
# Apply cleaning
df['hotel_name_clean'] = df['hotel_name'].apply(simple_clean)
df['address_clean'] = df['address_line1'].apply(simple_clean)
df['city_clean'] = df['city'].apply(simple_clean)
df['country_clean'] = df['country'].apply(simple_clean)

In [24]:
# Show before/after
print("Cleaned vs Original (first 3 rows):")
for i in range(3):
    print(f"Row {i}:")
    print(f"  Original: {df.loc[i, 'hotel_name']}")
    print(f"  Cleaned:  {df.loc[i, 'hotel_name_clean']}")
    print(f"  City Orig: {df.loc[i, 'city']} -> Clean: {df.loc[i, 'city_clean']}")
    print()

Cleaned vs Original (first 3 rows):
Row 0:
  Original: Marriott Bangalore Outer Ring Road
  Cleaned:  marriott bangalore outer ring road
  City Orig: Bangalore -> Clean: bangalore

Row 1:
  Original: Marriott ORR Blr - Outer Ring Rd
  Cleaned:  marriott orr blr outer ring rd
  City Orig: Bengaluru -> Clean: bengaluru

Row 2:
  Original: Hotel Marriot Bengaluru ORR
  Cleaned:  hotel marriot bengaluru orr
  City Orig: Bangalore -> Clean: bangalore



### 5. Similarity Scoring Model

In [25]:
#Using fuzzywuzzy to compute a string similarity score

from fuzzywuzzy import fuzz

def fuzzy_similarity(str1, str2):
    return fuzz.ratio(str1, str2) / 100.0

# Example
print("Fuzzy match examples:")
print(f"bangalore vs bengaluru: {fuzzy_similarity('bangalore', 'bengaluru')}")
print(f"road vs rd: {fuzzy_similarity('road', 'rd')}")
print(f"marriott vs marriot: {fuzzy_similarity('marriott', 'marriot')}")

Fuzzy match examples:
bangalore vs bengaluru: 0.67
road vs rd: 0.67
marriott vs marriot: 0.93


D:\My Work\PythonVsCode\Projects\ML-DL\Machine learining prac\MLDL\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [26]:
print(df['city_clean'].unique())

['bangalore' 'bengaluru' 'london' 'new york' 'paris' 'tokyo' 'udaipur'
 'sydney' 'manali' 'seoul' 'rome' 'doha' 'cape town' 'auckland' 'vienna'
 'montreal' 'chicago' 'hanoi' 'copenhagen' 'rio de janeiro' 'mexico city'
 'lisbon' 'brussels' 'istanbul' 'los angeles']


In [27]:
from fuzzywuzzy import fuzz

def fuzzy_match(str1, str2):
    return fuzz.ratio(str1, str2) / 100.0

#created a function to add more cities in future
def normalize_city(city):
    """Map known city variations"""
    city = city.lower().strip()
    city_map = {
        'bangalore': 'bengaluru',
        'bengaluru': 'bengaluru'  # standardize
    }
    return city_map.get(city, city)


df['city_normalized'] = df['city_clean'].apply(normalize_city)

In [28]:
df.head()

,record_id,source_system,hotel_name,address_line1,address_line2,city,state,country,postal_code,phone_number,phone_clean,hotel_name_clean,address_clean,city_clean,country_clean,city_normalized
0,R001,GDS_A,Marriott Bangalore Outer Ring Road,Outer Ring Road,Doddanekundi,Bangalore,Karnataka,IN,560103,-12345667,12345667,marriott bangalore outer ring road,outer ring road,bangalore,in,bengaluru
1,R002,OTA_B,Marriott ORR Blr - Outer Ring Rd,Outer Ring Rd,NaN,Bengaluru,Karnataka,India,560103,-12345667,12345667,marriott orr blr outer ring rd,outer ring rd,bengaluru,india,bengaluru
2,R003,DIRECT_C,Hotel Marriot Bengaluru ORR,Outer Ring Road,NaN,Bangalore,KA,IN,560103,-12345667,12345667,hotel marriot bengaluru orr,outer ring road,bangalore,in,bengaluru
3,R004,GDS_A,Holiday Inn Express Whitefield,ITPL Main Road,NaN,Bangalore,Karnataka,IN,560066,-99998877,99998877,holiday inn express whitefield,itpl main road,bangalore,in,bengaluru
4,R005,OTA_B,Holiday Inn Exp. Bengaluru Whitefield,ITPL Rd,Whitefield,Bengaluru,KA,India,560066,-99998877,99998877,holiday inn exp bengaluru whitefield,itpl rd,bengaluru,india,bengaluru


In [29]:
def hotel_similarity(row1, row2):
    """
    Calculate similarity score between two hotel records.
    Returns: float between 0 and 1
    """
    score = 0
    weights = {
        'name': 0.30,
        'address': 0.20,
        'city': 0.15,
        'postal': 0.15,
        'phone': 0.10,
        'country': 0.10
    }
    
    # 1. Hotel Name (fuzzy)
    name_sim = fuzzy_match(row1['hotel_name_clean'], row2['hotel_name_clean'])
    
    # 2. Address (fuzzy)
    addr_sim = fuzzy_match(row1['address_clean'], row2['address_clean'])
    
    # 3. City (normalized exact)
    city_match = 1 if row1['city_normalized'] == row2['city_normalized'] else 0
    
    # 4. Postal Code (exact)
    postal_match = 1 if row1['postal_code'] == row2['postal_code'] else 0
    
    # 5. Phone (exact on cleaned)
    phone_match = 1 if row1['phone_clean'] == row2['phone_clean'] else 0
    
    # 6. Country (exact on cleaned)
    country_match = 1 if row1['country_clean'] == row2['country_clean'] else 0
    
    # Weighted sum
    score = (name_sim * weights['name'] +
             addr_sim * weights['address'] +
             city_match * weights['city'] +
             postal_match * weights['postal'] +
             phone_match * weights['phone'] +
             country_match * weights['country'])
    
    return round(score, 3)

# Test on known duplicates
test_idx1, test_idx2 = 0, 1  # Marriott Bangalore duplicates
sim_score = hotel_similarity(df.iloc[test_idx1], df.iloc[test_idx2])

print("Test - Known duplicate pair:")
print(f"Hotel 1: {df.iloc[test_idx1]['hotel_name']}")
print(f"Hotel 2: {df.iloc[test_idx2]['hotel_name']}")
print(f"Similarity score: {sim_score}")
print(f"Verdict: {'SAME' if sim_score > 0.7 else 'DIFFERENT'}")

Test - Known duplicate pair:
Hotel 1: Marriott Bangalore Outer Ring Road
Hotel 2: Marriott ORR Blr - Outer Ring Rd
Similarity score: 0.82
Verdict: SAME


In [30]:
# Testing on obviously different hotels
test_idx3, test_idx4 = 0, 10  # Marriott Bangalore vs Courtyard NY
sim_score2 = hotel_similarity(df.iloc[test_idx3], df.iloc[test_idx4])

print("\nTest - Different hotels:")
print(f"Hotel 1: {df.iloc[test_idx3]['hotel_name']} ({df.iloc[test_idx3]['city']})")
print(f"Hotel 2: {df.iloc[test_idx4]['hotel_name']} ({df.iloc[test_idx4]['city']})")
print(f"Similarity score: {sim_score2}")
print(f"Verdict: {'SAME' if sim_score2 > 0.7 else 'DIFFERENT'}")


Test - Different hotels:
Hotel 1: Marriott Bangalore Outer Ring Road (Bangalore)
Hotel 2: Courtyard NYC Times Sq (New York)
Similarity score: 0.064
Verdict: DIFFERENT


### 6. Clustering for Canonical IDs

In [33]:
# Creating similarity matrix

n = len(df)
sim_matrix = np.eye(n)  # diagonal = 1 (same hotel)

print("Building similarity matrix...")
for i in range(n):
    for j in range(i+1, n):
        score = hotel_similarity(df.iloc[i], df.iloc[j])
        sim_matrix[i][j] = score
        sim_matrix[j][i] = score

# Convert similarity to distance (1 - similarity)
distance_matrix = 1 - sim_matrix

# Cluster (eps = 0.3 means require similarity > 0.7 to cluster)
clustering = DBSCAN(metric='precomputed', eps=0.3, min_samples=1)
clusters = clustering.fit_predict(distance_matrix)

# Assign canonical IDs
df['canonical_hotel_id'] = ['HOTEL_' + str(cid) for cid in clusters]

print(f"\nCreated {len(set(clusters))} canonical hotel groups")


Building similarity matrix...

Created 25 canonical hotel groups


In [34]:
# Printing some clusters

print("Sample clusters:")
cluster_sample = df[['hotel_name', 'city', 'canonical_hotel_id']].head(15)
print(cluster_sample.to_string(index=False))

Sample clusters:
                           hotel_name      city canonical_hotel_id
   Marriott Bangalore Outer Ring Road Bangalore            HOTEL_0
     Marriott ORR Blr - Outer Ring Rd Bengaluru            HOTEL_0
          Hotel Marriot Bengaluru ORR Bangalore            HOTEL_0
       Holiday Inn Express Whitefield Bangalore            HOTEL_1
Holiday Inn Exp. Bengaluru Whitefield Bengaluru            HOTEL_1
   Holiday Inn Express Blr-Whitefield Bangalore            HOTEL_1
       Hilton London Heathrow Airport    London            HOTEL_2
                        Hilton LHR T4    London            HOTEL_2
      Hilton London Heathrow T4 Hotel    London            HOTEL_2
      Courtyard New York Times Square  New York            HOTEL_3
               Courtyard NYC Times Sq  New York            HOTEL_3
Courtyard by Marriott Times Square NY  New York            HOTEL_3
          Ibis Paris Montmartre 18ème     Paris            HOTEL_4
           ibis Paris Montmartre 18th     Par

### 7. Matching Service for New Hotels

In [35]:
def match_hotel(new_hotel_dict, threshold=0.7):
    """
    new_hotel_dict: dict with keys like hotel_name, address_line1, city, etc.
    Returns: canonical_hotel_id, match_score, status
    """
    # Clean the new hotel data
    new_hotel_clean = {
        'hotel_name_clean': simple_clean(new_hotel_dict.get('hotel_name', '')),
        'address_clean': simple_clean(new_hotel_dict.get('address_line1', '')),
        'city_normalized': normalize_city(simple_clean(new_hotel_dict.get('city', ''))),
        'postal_code': str(new_hotel_dict.get('postal_code', '')),
        'phone_clean': str(abs(int(new_hotel_dict.get('phone_number', 0)))),
        'country_clean': simple_clean(new_hotel_dict.get('country', ''))
    }
    
    best_score = 0
    best_cluster_id = None
    
    # Compare with existing canonical clusters (use first record from each cluster as representative)
    cluster_representatives = {}
    for cluster_id in df['canonical_hotel_id'].unique():
        cluster_hotels = df[df['canonical_hotel_id'] == cluster_id]
        rep = cluster_hotels.iloc[0]  # first hotel in cluster as representative
        cluster_representatives[cluster_id] = rep
    
    # Compare with each cluster representative
    for cluster_id, rep in cluster_representatives.items():
        # Convert rep to dict for similarity function
        rep_dict = {
            'hotel_name_clean': rep['hotel_name_clean'],
            'address_clean': rep['address_clean'],
            'city_normalized': rep['city_normalized'],
            'postal_code': str(rep['postal_code']),
            'phone_clean': rep['phone_clean'],
            'country_clean': rep['country_clean']
        }
        
        # Create temporary rows for similarity calculation
        temp_df = pd.DataFrame([rep_dict, new_hotel_clean])
        score = hotel_similarity(temp_df.iloc[0], temp_df.iloc[1])
        
        if score > best_score:
            best_score = score
            best_cluster_id = cluster_id
    
    # Decision
    if best_score >= threshold:
        return {
            'canonical_hotel_id': best_cluster_id,
            'match_score': best_score,
            'status': 'MATCHED',
            'matched_hotel_name': df[df['canonical_hotel_id'] == best_cluster_id].iloc[0]['hotel_name']
        }
    else:
        # Create new canonical ID
        new_id = f'HOTEL_NEW_{len(cluster_representatives)}'
        return {
            'canonical_hotel_id': new_id,
            'match_score': best_score,
            'status': 'NEW',
            'matched_hotel_name': None
        }



In [36]:
# Test with a known duplicate
test_new_hotel = {
    'hotel_name': 'Marriott Outer Ring Road Bangalore',
    'address_line1': 'Outer Ring Road',
    'city': 'Bengaluru',
    'postal_code': '560103',
    'phone_number': -12345667,
    'country': 'India'
}

result = match_hotel(test_new_hotel)
print("Test match for known duplicate:")
print(result)

Test match for known duplicate:
{'canonical_hotel_id': 'HOTEL_0', 'match_score': 0.813, 'status': 'MATCHED', 'matched_hotel_name': 'Marriott Bangalore Outer Ring Road'}


In [40]:
# Test with brand new hotel (it should create new ID)
test_new_hotel_2 = {
    'hotel_name': 'Grand Hyatt Mumbai',
    'address_line1': 'Bandra Kurla Complex',
    'city': 'Mumbai',
    'postal_code': '400051',
    'phone_number': 22223333,
    'country': 'India'
}

result2 = match_hotel(test_new_hotel_2)
print("\nTest match for brand new hotel:")
print(result2)


Test match for brand new hotel:
{'canonical_hotel_id': 'HOTEL_NEW_25', 'match_score': 0.169, 'status': 'NEW', 'matched_hotel_name': None}


### 8. Summary & Next Steps
- Working prototype with rule-based matching
- Production-ready MLOps design (see separate document)